Imprtant libraries to install

```
pip install Flask
pip install Flask-WTF
pip install joblib
```

In [1]:
#imports 
import pandas as pd
import numpy as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

## Data and Model

In [50]:
ufc_fights = pd.read_csv('data/ufc_fight_data.csv')
ufc_stats = pd.read_csv('data/ufc_fight_stat_data.csv')
ufc_fighters = pd.read_csv('data/ufc_fighter_data.csv')

### Cleaning the data

In [51]:
# Create full name column and drop unnecessary columns
ufc_fighters['fighter_fullname'] = ufc_fighters['fighter_f_name'] + ' ' + ufc_fighters['fighter_l_name']
ufc_fighters.drop(['fighter_f_name', 'fighter_l_name', 'fighter_dob'], axis=1, inplace=True)

# Join datasets to get fight and fighter stats for each fight
ufc_df = ufc_fights.merge(ufc_stats, on='fight_id', how='outer') \
                   .merge(ufc_fighters, on='fighter_id', how='outer')

# Remove URL columns as they are not relevant
ufc_df.drop(['fighter_url'], axis=1, inplace=True)

# Replace 'NULL' and '--' in 'ctrl_time' and 'finish_time' columns with '0:00'
ufc_df['ctrl_time'] = ufc_df['ctrl_time'].replace(['NULL', '--'], '0:00')
ufc_df['finish_time'] = ufc_df['finish_time'].replace(['NULL', '--'], '0:00')

# Split 'finish_time' and 'ctrl_time' columns
finish_split = ufc_df['finish_time'].str.split(':', expand=True)
ctrl_split = ufc_df['ctrl_time'].str.split(':', expand=True)

# Convert 'finish_time' to total seconds
# Handle NaN values by filling with '0' or appropriate placeholder
finish_split = finish_split.fillna(0)
ufc_df['finish_time'] = (60 * finish_split[0].astype(float).fillna(0).astype(int) +
                         finish_split[1].astype(float).fillna(0).astype(int) + 
                         300 * ufc_df['finish_round'].astype(float).fillna(0).astype(int))

# Convert 'ctrl_time' to total seconds
# Handle NaN values by filling with '0' or appropriate placeholder
ctrl_split = ctrl_split.fillna(0)
ufc_df['ctrl_time'] = (60 * ctrl_split[0].astype(float).fillna(0).astype(int) +
                       ctrl_split[1].astype(float).fillna(0).astype(int))


In [49]:
display(ufc_df.head())

,fight_id,event_id,referee,f_1,f_2,winner,num_rounds,title_fight,weight_class,gender,...,fighter_nickname,fighter_height_cm,fighter_weight_lbs,fighter_reach_cm,fighter_stance,fighter_w,fighter_l,fighter_d,fighter_nc_dq,fighter_fullname
0,7218.0,664.0,Herb Dean,2976.0,2884.0,2884.0,5,F,Lightweight,M,...,Ataman,172.72,155.0,180.34,Switch,12.0,3.0,0.0,NaN,Rafael Fiziev
1,6922.0,639.0,Marc Goddard,2902.0,2976.0,2902.0,3,F,Lightweight,M,...,Ataman,172.72,155.0,180.34,Switch,12.0,3.0,0.0,NaN,Rafael Fiziev
2,6585.0,611.0,Mark Smith,3185.0,2976.0,2976.0,5,F,Lightweight,M,...,Ataman,172.72,155.0,180.34,Switch,12.0,3.0,0.0,NaN,Rafael Fiziev
3,6279.0,586.0,Herb Dean,1042.0,2976.0,2976.0,3,F,Lightweight,M,...,Ataman,172.72,155.0,180.34,Switch,12.0,3.0,0.0,NaN,Rafael Fiziev
4,6110.0,572.0,Jacob Montalvo,2728.0,2976.0,2976.0,3,F,Lightweight,M,...,Ataman,172.72,155.0,180.34,Switch,12.0,3.0,0.0,NaN,Rafael Fiziev


In [37]:
# Replace 'NULL' in 'ctrl_time' column with '0:00'
ufc_df['ctrl_time'] = ufc_df['ctrl_time'].replace('NULL', '0:00')

# Split 'finish_time' and 'ctrl_time' columns
finish_split = ufc_df['finish_time'].str.split(':', expand=True)
ctrl_split = ufc_df['ctrl_time'].str.split(':', expand=True)

# Convert 'finish_time' to total seconds
# Handle NaN values by filling with '0' or appropriate placeholder
finish_split = finish_split.fillna(0)
ufc_df['finish_time'] = (60 * finish_split[0].astype(float).fillna(0).astype(int) +
                     finish_split[1].astype(float).fillna(0).astype(int) + 
                     300 * ufc_df['finish_round'].astype(float).fillna(0).astype(int))

# Convert 'ctrl_time' to total seconds
# Handle NaN values by filling with '0' or appropriate placeholder
ctrl_split = ctrl_split.fillna(0)
ufc_df['ctrl_time'] = (60 * ctrl_split[0].astype(float).fillna(0).astype(int) +
                   ctrl_split[1].astype(float).fillna(0).astype(int))

# df now has 'finish_time' and 'ctrl_time' as total seconds


AttributeError: Can only use .str accessor with string values!

In [2]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)
sample_df.head()

,wid,reflection_period,original_hm,moment,modified,num_sentence,target,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [3]:
sample_df["target"].value_counts()

target
affection           4810
achievement         4276
bonding             1750
enjoy_the_moment    1514
leisure             1306
nature               252
exercise             217
Name: count, dtype: int64

In [4]:
train_df, test_df = train_test_split(sample_df, test_size=0.2, random_state=123)
X_train_happy, y_train_happy = train_df["moment"], train_df["target"]
X_test_happy, y_test_happy = test_df["moment"], test_df["target"]

In [5]:
pipe_lr = make_pipeline(
    CountVectorizer(stop_words="english"),
    LogisticRegression(max_iter=2000),
)

In [6]:
pipe_lr.fit(X_train_happy, y_train_happy);

Saving the model:

In [7]:
import joblib

with open("web_application/moment_predictor.joblib", "wb") as f:
    joblib.dump(pipe_lr, f)

## Running the application

In [10]:
%run -i "web_application/app.py"

predictform
 * Serving Flask app 'app'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/May/2024 20:34:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2024 20:34:42] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2024 20:34:42] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/May/2024 20:34:52] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [18/May/2024 20:34:52] "GET /prediction HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2024 20:34:52] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [18/May/2024 20:34:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2024 20:34:55] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [18/May/2024 20:35:15] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [18/May/2024 20:35:15] "GET /prediction HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2024 20:35:15] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [18/May/2024 20:35:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2024 20:35:20] "GET /static/style.css HTTP/1.1" 304 -
